In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import os # 因為要建立資料夾
from tensorflow.keras.models import Sequential # CNN 建立空模函式
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D # CNN 函式
from tensorflow.python.keras.utils import np_utils # 層

In [2]:
# 掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 載入各個資料夾

In [3]:
import os
path = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/train'
files = os.listdir(path)
directories = list()

In [4]:
len(files)

29

In [5]:
for dir in files:
  if os.path.isdir(os.path.join(path, dir)):
    directories.append(dir)
directories

['哈密瓜',
 '櫻桃',
 '檸檬',
 '梨',
 '小黃瓜',
 '柚子',
 '楊梅',
 '山竹',
 '椰子',
 '榴槤',
 '奇異果',
 '火龍果',
 '白葡萄',
 '芒果',
 '石榴',
 '聖女番茄',
 '砂糖橘',
 '胡蘿蔔',
 '紅葡萄',
 '紅番茄',
 '苦瓜',
 '紅蘋果',
 '草莓',
 '鳳梨',
 '西瓜',
 '香蕉',
 '青蘋果',
 '龍眼',
 '荔枝']

# 分類

## 製成縮圖函式

In [6]:
def resizeFile(dir_path, img_files): # dir_path 某個水果資料夾 img_files 資料夾中的所有檔案
  size = (250, 250)
  for item in img_files:

    item = os.path.join(dir_path, item)

    images = cv2.imread(item)
    # :2 兩通道
    h, w = images.shape[:2]
    # 找最大邊
    if h > size[0] or w > size[1]:
      # 找出最大比例 => 等比例縮小
      ratio = max(h/size[0], w/size[1])
      images = cv2.resize(images, (int(w/ratio), int(h/ratio)))
      # 重新抓取新的寬、高
    h, w = images.shape[:2]
    # 高度的黑邊
    pad_h = size[0]-h
    # 寬度的黑邊
    pad_w = size[1]-w
    # 上、下補黑邊的高度
    top, bottom = pad_h//2, pad_h-(pad_h//2)
    # 左、右補黑邊的寬度
    left, right = pad_w//2, pad_w-(pad_w//2)
    # 補黑邊(填充：適用於圖片縮小時)
    images = cv2.copyMakeBorder(images, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
    # 抓檔名
    name = item.split('/')[-1]
    # 用程式判斷是否有所需的資料夾

    cv2.imwrite(f"{dir_path}/resize/{name}", images)

In [7]:
x_train = list()
y_train = list()

def create_data(path, category):
  for p in os.listdir(path):
    # 轉灰階
    img_array = cv2.imread(os.path.join(path, p), 0)
    # 訓練題目
    x_train.append(img_array)
    # 訓練答案
    y_train.append(category)

In [8]:
category = 0
for directory in directories:
  dir_path = os.path.join(path, directory)
  # 目錄中的所有檔案
  dir_file = os.listdir(dir_path)
  img_files = list()
  for file in dir_file:
    # 如附檔名為這三種的任何一種
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
      img_files.append(file)
  resizeDir = os.path.join(dir_path, 'resize')
  if not os.path.exists(resizeDir):
    os.makedirs(resizeDir)
    resizeFile(dir_path, img_files)
  # 分類
  create_data(resizeDir, category)
  category += 1


In [13]:
len(x_train)

4186

In [ ]:
len(x_train)

4186

# 打散成組的訓練資料

In [11]:
# reshape(-1,250,250,1) 一組資料一維改成一個資料一維
x_train = np.array(x_train).reshape(-1,250,250,1) # (張數, 長, 寬, 通道數)
y_train = np.array(y_train)

x_train = x_train / 255 # 色階 => 收斂
# 將答案分類 OneHot
y_train = np_utils.to_categorical(y_train)

In [ ]:
len(y_train[0])

29

# 建模

In [ ]:
model = Sequential()

In [ ]:
# filters=32 特徵圖 => 濾波器數目，就是下圖的每一階段的輸出面數或是深度，輸出的圖稱為『特徵圖』(Feature Map)，通常是4的倍數。
# kernel_size=(3,3) 卷機核的大小
# padding='same' 超過邊界時一律補0
# input_shape=x_train.shape[1:] 從索引值1開始抓到最後(張數不要)
# activation='relu' 修正線性單元激勵函式 (把負數變為0)

# 卷積
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
# 池化
model.add(MaxPooling2D(pool_size=(2,2))) # pool_size=(2,2) 為預設，不寫也可以
# 卷積
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
# 池化
model.add(MaxPooling2D(pool_size=(2,2)))
# 卷積
#model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
# 池化
#model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
# 放棄的比率
model.add(Dropout(0.2))
# 扁平化
model.add(Flatten())
# 放棄的比率
model.add(Dropout(0.2))
# 連接層：通常用1024(三通道 32*32*32)
model.add(Dense(1024, activation='relu'))
# 輸出層：29種分類
model.add(Dense(29, activation='softmax'))

# 編譯(程式)

In [ ]:
# optimizer='adam' 優化器
# loss 損失函式 ='mse' 平均方差
# metrics='accurancy' 只顯示正確度
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 打亂訓練資料群

In [ ]:
from sklearn.model_selection import train_test_split
rX_train, rX_test, ry_train, ry_test = train_test_split(x_train, y_train, test_size=0.2, random_state=1)

# 訓練

## 第一種訓練方式

In [ ]:
model.fit(rX_train, ry_train, epochs=10, batch_size=50, validation_split=0.25)

Epoch 1/10
63/63 [==============================] - 24s 246ms/step - loss: 4.0683 - accuracy: 0.0446 - val_loss: 3.2967 - val_accuracy: 0.1213
Epoch 2/10
63/63 [==============================] - 12s 196ms/step - loss: 3.0446 - accuracy: 0.1472 - val_loss: 3.0331 - val_accuracy: 0.1270
Epoch 3/10
63/63 [==============================] - 13s 200ms/step - loss: 2.2949 - accuracy: 0.3658 - val_loss: 3.0402 - val_accuracy: 0.1748
Epoch 4/10
63/63 [==============================] - 13s 200ms/step - loss: 1.2233 - accuracy: 0.6673 - val_loss: 3.3503 - val_accuracy: 0.1719
Epoch 5/10
63/63 [==============================] - 13s 202ms/step - loss: 0.4530 - accuracy: 0.8878 - val_loss: 3.9744 - val_accuracy: 0.1777
Epoch 6/10
63/63 [==============================] - 13s 202ms/step - loss: 0.1733 - accuracy: 0.9611 - val_loss: 4.4164 - val_accuracy: 0.1748
Epoch 7/10
63/63 [==============================] - 13s 204ms/step - loss: 0.0855 - accuracy: 0.9822 - val_loss: 4.7881 - val_accuracy: 0.1824

# 測試

## 製作測試的資料

In [ ]:
import os
test_path = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/photos/水果30分類/test'
test_files = os.listdir(test_path)
test_directories = list()

In [ ]:
len(test_files)

29

In [ ]:
for dir in test_files:
  if os.path.isdir(os.path.join(test_path, dir)):
    test_directories.append(dir)
test_directories

['梨',
 '奇異果',
 '小黃瓜',
 '哈密瓜',
 '楊梅',
 '椰子',
 '柚子',
 '山竹',
 '榴槤',
 '櫻桃',
 '紅番茄',
 '红葡萄',
 '砂糖橘',
 '石榴',
 '檸檬',
 '红蘋果',
 '白葡萄',
 '火龍果',
 '聖女番茄',
 '胡蘿蔔',
 '西瓜',
 '荔枝',
 '苦瓜',
 '青蘋果',
 '草莓',
 '鳳梨',
 '香蕉',
 '龍眼',
 '芒果']

## 分類

In [ ]:
x_test = list()
y_test = list()

def create_testdata(path, category):
  for p in os.listdir(path):
    # 轉灰階
    img_array = cv2.imread(os.path.join(path, p), 0)
    # 訓練題目
    x_test.append(img_array)
    # 訓練答案
    y_test.append(category)

In [ ]:
test_category = 0
for directory in test_directories:
  dir_path = os.path.join(test_path, directory)
  # 目錄中的所有檔案
  dir_file = os.listdir(dir_path)
  img_files = list()
  for file in dir_file:
    # 如附檔名為這三種的任何一種
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
      img_files.append(file)
  resizeDir = os.path.join(dir_path, 'resize')
  if not os.path.exists(resizeDir):
    os.makedirs(resizeDir)
    resizeFile(dir_path, img_files)
  # 分類
  create_testdata(resizeDir, test_category)
  test_category += 1

## 打散成組的測試資料

In [ ]:
# reshape(-1,250,250,1) 一組資料一維改成一個資料一維
x_test = np.array(x_test).reshape(-1,250,250,1) # (張數, 長, 寬, 通道數)
y_test = np.array(y_test)

x_test = x_test / 255 # 色階 => 收斂
# 將答案分類 OneHot
y_test = np_utils.to_categorical(y_test)

In [ ]:
raX_train, raX_test, ray_train, ray_test = train_test_split(x_test, y_test, test_size=0.2, random_state=1)

# 測試

In [ ]:
predict1 = model.predict(rX_train)
predict2 = model.predict(raX_train)

32/32 [==============================] - 1s 19ms/step


In [ ]:
right = 0
for i in range(len(predict1)):
  ans = np.argmax(predict1[i])
  pre = ry_train[i][ans]
  if pre == 1:
    right += 1
right / len(predict1)

0.04613841524573721

In [ ]:
right = 0
for i in range(len(predict2)):
  ans = np.argmax(predict2[i])
  pre = ray_train[i][ans]
  if pre == 1:
    right += 1
right / len(predict2)

0.04618473895582329

## 第二種訓練方式

In [ ]:
model.fit(rX_train, ry_train, epochs=10, batch_size=50, validation_data=(x_test, y_test))